In [53]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pandas_market_calendars as mcal
from importlib import reload
import sys
import os
import time

In [16]:
import functions; functions = reload(functions)
from functions import *

In [3]:
apikey = '3CenRhJBzNqh2_C_5S38pOyt3ozLvQDm'

In [4]:
# todo
# 1. test function for daily aggregates - none/errors/edge cases/before available
# 2. design pipeline for many jsons to csv
# 3. .db maybe?
# 4. if not, make csv
# 5. update/remake .py files to download and merge missing

In [5]:
date = '2020-01-01'
url = f'https://api.polygon.io/v2/aggs/grouped/locale/us/market/stocks/{date}?apiKey={apikey}'

In [6]:
response = requests.get(url)

In [7]:
response.json()

{'queryCount': 0,
 'resultsCount': 0,
 'adjusted': True,
 'status': 'OK',
 'request_id': 'a426fcea4aa5004a97f7d0e49adcb888'}

In [50]:
unauth = daily_agg('2018-01-02')
holiday = daily_agg('2020-01-01')
good = daily_agg('2020-06-01')

Attempted to request data past historical entitlements. Please upgrade your plan at https://polygon.io/pricing
No data available for 2020-01-01


In [17]:
mon = daily_agg('2024-06-03')
tue = daily_agg('2024-06-04')
wed = daily_agg('2024-06-05')

In [18]:
alljson = []
for json in [mon, tue, wed]:
    alljson.extend(json)

In [13]:
trd = get_trading_days()

In [14]:
len(trd)

1259

In [15]:
trd[0]

'2019-06-17'

In [ ]:
for i in range(100):
    time.sleep(0.1)
    print()

In [60]:
from IPython.display import clear_output, display

In [65]:
from tqdm import tqdm

In [66]:
def timer_test(n = 100):
    #progress = display(f'0/{n} (percentage: 0%)')
    for i in tqdm(range(n)):
        time.sleep(0.1)
        

In [73]:
def datelist_to_df(datelist):
    alljson = []
    for i, date in enumerate(tqdm(datelist)):
        data = daily_agg(date)
        if data is not None:
            alljson.extend(data)
    
    return pd.DataFrame(alljson)

In [72]:
datelist_to_df2(get_trading_days(from_date='2024-05-01'))

100%|██████████| 32/32 [00:26<00:00,  1.20it/s]

Attempted to request today's data before end of day. Please upgrade your plan at https://polygon.io/pricing


In [22]:
thisyear = get_trading_days(from_date='2024-01-01')

In [23]:
year = datelist_to_df(thisyear)

Attempted to request today's data before end of day. Please upgrade your plan at https://polygon.io/pricing


In [32]:
a = datetime.now() 

In [33]:
b = datetime.now()

In [40]:
a.utcfromtimestamp

<function datetime.utcfromtimestamp>

In [39]:
a.hour

2